In [1]:
from google.colab import drive, files as g_files

drive.mount('/proj')

Mounted at /proj


We will train a Darknet model to detect dolphin fins using Yolov4

In [2]:
# Imports 

import os
import random
import shutil

from glob import glob

In [3]:
# Constants

SEED = 100
TEST_PROP =  0.15

In [ ]:
# %cd /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/images
# !zip -r NDD20.zip NDD20

On Google Colab, it's a good idea to transfer your Google Drive contents onto the local VM. The VM accesses Drive files over the network, so network latency increases training times by quite a bit. In your local environment, you don't need to do this step.

In [4]:
!unzip /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/images/NDD20.zip -d /content

Archive:  /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/images/NDD20.zip
   creating: /content/NDD20/
  inflating: /content/NDD20/ABOVE_LABELS.json  
   creating: /content/NDD20/data/
  inflating: /content/NDD20/data/2085.txt  
  inflating: /content/NDD20/data/2079.txt  
  inflating: /content/NDD20/data/2084.txt  
  inflating: /content/NDD20/data/2088.txt  
  inflating: /content/NDD20/data/2086.txt  
  inflating: /content/NDD20/data/207.txt  
  inflating: /content/NDD20/data/2087.txt  
  inflating: /content/NDD20/data/2090.txt  
  inflating: /content/NDD20/data/2089.txt  
  inflating: /content/NDD20/data/2080.txt  
  inflating: /content/NDD20/data/2092.txt  
  inflating: /content/NDD20/data/2091.txt  
  inflating: /content/NDD20/data/208.txt  
  inflating: /content/NDD20/data/2081.txt  
  inflating: /content/NDD20/data/2093.txt  
  inflating: /content/NDD20/data/2094.txt  
  inflating: /content/NDD20/data/2095.txt  
  inflating: /content/NDD20/data/2096.txt  
  inflating: /conte

We'll build a list all of the available image files.

In [5]:
data_dir = "/content/NDD20/"
file_dir = f"{data_dir}data/"
files = glob(f'{file_dir}*.jpg')

In [6]:
files[0:5]

['/content/NDD20/data/1271.jpg',
 '/content/NDD20/data/1306.jpg',
 '/content/NDD20/data/94.jpg',
 '/content/NDD20/data/1494.jpg',
 '/content/NDD20/data/768.jpg']

Next, the files will be shuffled and split into the training and testing set. An 85-15 split is used. [From the DarkNet documentation, we want around at least 2000 training objects per class.](https://github.com/AlexeyAB/darknet)

In [7]:
def test_train_split(files, test_prop = 0.1):
    list_size = int(round(test_prop * len(files)))
    shuffled = files[:]
    random.seed(SEED)
    random.shuffle(shuffled)
    return shuffled[list_size:], shuffled[:list_size]

def to_relative_path(full_path_list):
  relative_path_list = []

  for file in full_path_list:
    split_file = file.split("/")
    relative_path = f'{split_file[-2]}/{split_file[-1]}'
    relative_path_list.append(relative_path)

  return relative_path_list

In [8]:
train, test = test_train_split(files, TEST_PROP)

In [9]:
print(f"Train length: {len(train)}\nTest length: {len(test)}\nTotal: {len(files)}")

Train length: 1871
Test length: 330
Total: 2201


From the output above, we could use a 90-10 split instead.

The lists of train and test file names will be written to train.txt and test.txt respectively.

In [10]:
with open("/tmp/train.txt", "w") as outfile:
    outfile.write("\n".join(train))

with open("/tmp/test.txt", "w") as outfile:
    outfile.write("\n".join(test))

shutil.move("/tmp/train.txt", data_dir + "train.txt")
shutil.move("/tmp/test.txt", data_dir + "test.txt")

'/content/NDD20/test.txt'

DarkNet needs to be compiled, and GPU support must be enabled. 

In [ ]:
#%cd /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/neural_nets/darknet/
#!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
#!sed -i 's/GPU=0/GPU=1/' Makefile
#!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
#!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
#!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

#!make

We can view the content of the DarkNet directory below.

In [11]:
%cd /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/neural_nets/darknet/
!ls

/proj/MyDrive/158780_Project_Dolphin_Computer_Vision/neural_nets/darknet
3rdparty		  DarknetConfig.cmake.in  net_cam_v3.sh
backup			  darknet_images.py	  net_cam_v4.sh
bad_label.list		  darknet.py		  obj
bad.list		  darknet_video.py	  README.md
build			  data			  results
build.ps1		  image_yolov3.sh	  scripts
cfg			  image_yolov4.sh	  src
chart.png		  include		  uselib
chart_yolov4-dolphin.png  json_mjpeg_streams.sh   vcpkg.json
cmake			  libdarknet.so		  video_yolov3.sh
CMakeLists.txt		  LICENSE		  video_yolov4.sh
darknet			  Makefile


In [12]:
%env IMAGES_DIR=/content/NDD20
%env MODEL_DIR=/proj/MyDrive/158780_Project_Dolphin_Computer_Vision/models/darknet

env: IMAGES_DIR=/content/NDD20
env: MODEL_DIR=/proj/MyDrive/158780_Project_Dolphin_Computer_Vision/models/darknet


Next, we'll begin the training process.

In [ ]:
! chmod +x ./darknet
! chmod +rwx /proj/MyDrive/158780_Project_Dolphin_Computer_Vision/models/darknet/training
#! ./darknet detector train ${IMAGES_DIR}/obj.data ${MODEL_DIR}/yolov4-dolphin.cfg ${MODEL_DIR}/yolov4.conv.137 -dont_show -map
! ./darknet detector train ${IMAGES_DIR}/obj.data ${MODEL_DIR}/yolov4-dolphin.cfg ${MODEL_DIR}/training/yolov4-dolphin_last.weights -dont_show -map

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 700, cudnn_half = 1, GPU: Tesla V100-SXM2-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Layer: 4,